In [2]:
import raster_geometry as rg
import numpy as np
import matplotlib.pyplot as plt
from skimage.transform import rotate
import pickle
import sys
from skimage.transform import rescale, resize, downscale_local_mean
sys.path.insert(0,'/home/georgeos/Documents/GitHub/SYMPTOMM2')
import itertools
from joblib import Parallel, delayed
from skimage.morphology import opening
from PIL import Image       
import pymunk
from skimage.transform import PiecewiseAffineTransform, warp
from skimage import data
from copy import deepcopy
from tqdm import tqdm
import pandas as pd
from skimage import draw
#import napari
from itertools import combinations
from SYMPTOMM import PSF
from matplotlib_scalebar.scalebar import ScaleBar
from cupyx.scipy.ndimage import convolve as cuconvolve
import tifffile
from skimage.exposure import match_histograms
import cupy as cp
from scipy.optimize import dual_annealing, shgo
from skimage.transform import resize
from skimage.metrics import structural_similarity as ssim
from scipy.optimize import basinhopping
import image_similarity_measures
from image_similarity_measures.quality_metrics import rmse, psnr, fsim, issm, sre, sam, uiq
from SYMPTOMM.general_drawing import *

In [3]:
with open("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/output_pickles/cell_timeseries_short_trench.p", "rb") as f:
    cell_timeseries = pickle.load(f)
with open("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/output_pickles/space_timeseries_short_trench.p", "rb") as f:
    space = pickle.load(f)


In [5]:
ID_props = generate_curve_props(cell_timeseries)
cell_timeseries_properties = Parallel(n_jobs=14)(delayed(gen_cell_props_for_draw)(a, ID_props) for a in cell_timeseries)

In [6]:
do_transformation = False

In [ ]:
scenes = Parallel(n_jobs=14)(delayed(draw_scene)(cell_properties, False) for cell_properties in tqdm(cell_timeseries_properties))


  4%|▎         | 14/399 [00:00<00:04, 87.51it/s]

In [ ]:
output_dir = "/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/training_data/OPL_renders"
_ = Parallel(n_jobs=10)(delayed(scene_plotter)(scenes[x][1],output_dir,"OPL_mask",x+4000,matplotlib_draw=False) for x in range(len(scenes)))
_ = Parallel(n_jobs=10)(delayed(scene_plotter)(scenes[x][0],output_dir,"OPL_intensity",x+4000,matplotlib_draw=False) for x in range(len(scenes)))

# Phase Contrast

In [ ]:
##From here, prototyping phase contrast
def get_trench_segments(space):
    trench_shapes = []
    for shape, body in zip(space.shapes, space.bodies):
        if body.body_type == 2:
            trench_shapes.append(shape)
    return trench_shapes

trench_segment_props = []
for x in get_trench_segments(space):
    trench_segment_props.append([x.bb,x.area, x.a, x.b])
    
trench_segment_props = pd.DataFrame(trench_segment_props)
trench_segment_props.columns = ["bb", "area", "a", "b"]
main_segments = trench_segment_props.sort_values("area",ascending=False).iloc[0:2]

In [ ]:
main_segments = main_segments.reindex(index=main_segments.index[::-1]) # run if you get a zero-width image output


In [ ]:
offset = 50

trench_multiplier = 30
cell_multiplier = 2
background_multiplier = 10
grid_def = [range(15,trench_multiplier), np.arange(0.1,cell_multiplier,0.1), range(background_multiplier)]
grid = list(itertools.product(*grid_def))
def generate_PC_OPL(trench_multiplier,cell_multiplier,background_multiplier):
    segment_1_top_left = (0 + offset, int(main_segments.iloc[0]["bb"][0] + offset))
    segment_1_bottom_right = (int(main_segments.iloc[0]["bb"][3] + offset), int(main_segments.iloc[0]["bb"][2] + offset))

    segment_2_top_left = (0 + offset, int(main_segments.iloc[1]["bb"][0] + offset))
    segment_2_bottom_right = (int(main_segments.iloc[1]["bb"][3] + offset), int(main_segments.iloc[1]["bb"][2] + offset))

    test_scene = np.zeros(scenes[300][0].shape) + background_multiplier
    rr, cc = draw.rectangle(start = segment_1_top_left, end = segment_1_bottom_right, shape = test_scene.shape)
    test_scene[rr,cc] = 1 * trench_multiplier
    rr, cc = draw.rectangle(start = segment_2_top_left, end = segment_2_bottom_right, shape = test_scene.shape)
    test_scene[rr,cc] = 1 * trench_multiplier

    circ_midpoint_y = (segment_1_top_left[1] + segment_2_bottom_right[1])/2
    radius = (segment_1_top_left[1] - offset - (segment_2_bottom_right[1] - offset))/2
    circ_midpoint_x = (offset) + radius

    rr, cc = draw.rectangle(start = segment_2_top_left, end = (circ_midpoint_x,80+segment_1_top_left[1] - offset), shape = test_scene.shape)
    test_scene[rr.astype(int),cc.astype(int)] = 1 * trench_multiplier

    rr, cc = draw.disk(center = (circ_midpoint_x, circ_midpoint_y), radius = radius, shape = test_scene.shape)
    rr_semi = rr[rr < (circ_midpoint_x + 1)]
    cc_semi = cc[rr < (circ_midpoint_x + 1)]
    test_scene[rr_semi,cc_semi] = background_multiplier
    test_scene += scenes[np.random.randint(len(scenes))][0] * cell_multiplier
    test_scene = test_scene[segment_2_top_left[0]:segment_1_bottom_right[0],segment_2_top_left[1]:segment_1_bottom_right[1]]

    expanded_scene = np.zeros((int(test_scene.shape[0]*1.2), test_scene.shape[1]*2)) + trench_multiplier
    expanded_scene[expanded_scene.shape[0] - test_scene.shape[0]:,int(test_scene.shape[1]/2):int(test_scene.shape[1]/2) + test_scene.shape[1]] = test_scene
    return expanded_scene

plt.figure(figsize=(10,10))
plt.imshow(generate_PC_OPL(30,1,10))

In [ ]:
import gc
gc.collect()

In [ ]:
#PC_OPLs = np.array([generate_PC_OPL(*k) for k in grid])
#viewer = napari.view_image(np.array(PC_OPLs), rgb=False)

In [ ]:
real_phase = tifffile.imread("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/top_trenches_PC/trench_{}/T_{}.tif".format(
    str(np.random.randint(56)).zfill(2),
    str(np.random.randint(20,25)).zfill(3)
))
plt.imshow(real_phase)

In [ ]:
def gaussian_2D(size, σ):
    x = np.linspace(0,size,size)
    μ = np.mean(x)
    A = 1/(σ*np.sqrt(2*np.pi))
    B = np.exp(-1/2 * (x-μ)**2/(σ**2))
    _gaussian_1D = A*B
    _gaussian_2D = np.outer(_gaussian_1D,_gaussian_1D)
    return _gaussian_2D

In [ ]:
condensers = {
    "Ph1": (0.45, 3.75, 24),
    "Ph2": (0.8, 5.0, 24),
    "Ph3": (1.0, 9.5, 24),
    "Ph4": (1.5, 14.0, 24),
    "PhF": (1.5, 19.0, 25)
} #W, R, Diameter

from scipy.special import jv, jve


def somb(x):
    z = np.zeros(x.shape)
    x = np.abs(x)
    idx = np.nonzero(x)
    z[idx] = 2*jv(1,np.pi*x[idx])/(np.pi*x[idx])
    return z

def get_phase_contrast_kernel(R,W,radius,scale,F,sigma):
    scale1 = 1000 # micron per millimeter
    F = F * scale1 # to microm
    Lambda = 0.55 # in micron % wavelength of light
    R = R * scale1 # to microm
    W = W * scale1 # to microm
    #The corresponding point spread kernel function for the negative phase contrast 

    meshgrid_arrange = np.arange(-radius,radius + 1,1)
    [xx,yy] = np.meshgrid(meshgrid_arrange,meshgrid_arrange)
    rr = np.sqrt(xx**2 + yy**2)*scale
    rr_dl = rr*(1/F)*(1/Lambda); # scaling with F and Lambda for dimension correction
    kernel1 = np.pi*R**2*somb(2*R*rr_dl);     
    kernel2 = np.pi*(R-W)**2*somb(2*(R-W)*rr_dl)


    kernel = kernel1 - 0*kernel2
    kernel = kernel/np.max(kernel)
    kernel[radius,radius] = kernel[radius,radius] + 1
    kernel = -kernel/np.sum(kernel)
    gaussian = gaussian_2D(radius*2+1, sigma)
    kernel = kernel * gaussian
    return kernel


W, R, diameter = condensers["Ph2"]
scale = 0.108379937 / 3 #0.35 #micron per pixel
min_sigma = 0.42*0.6/2 / scale # micron
kernel= get_phase_contrast_kernel(R, W, 50, scale, 5, 15)
scalebar = ScaleBar(scale, 'um')
plt.figure(figsize=(5.1,5.1))
plt.gca().add_artist(scalebar)
plt.imshow(kernel,cmap="Greys_r")
plt.axis("off")
plt.title("Phase Contrast")
plt.show()
kernel = cp.array(kernel)
kernel_np = kernel.get()
plt.figure(figsize = (15,10))
plt.plot(np.arange(-50,51)*scale,kernel_np[50,:])
plt.show()

In [ ]:
from skimage.util import random_noise
output = cuconvolve(cp.array(PC_OPLs[2000]),kernel)
output = output.get()
output = rescale(output, 1/4, anti_aliasing=False)[3:182,3:-3]
output = random_noise(output, mode="gaussian", mean=5,var=0.0000051,clip=False)
#viewer = napari.view_image(output_rescaled, rgb=False)

In [ ]:
top_segment = output
plt.figure(figsize=(5,5))
plt.imshow(top_segment,cmap="Greys_r")
top_segment.shape += (1,)
plt.show()

In [ ]:
def return_intersection_between_image_hists(img1, img2, bins):
    hist_1, _ = np.histogram(img1.flatten()/img1.flatten().max(), bins=bins)
    hist_2, _ = np.histogram(img2.flatten()/img2.flatten().max(), bins=bins)
    minima = np.minimum(hist_1, hist_2)
    intersection = np.true_divide(np.sum(minima), np.sum(hist_2))
    return intersection

In [ ]:
def convolve_rescale(image,kernel):
    output = cuconvolve(cp.array(image),cp.array(kernel))
    output = output.get()
    output = rescale(output, 1/4, anti_aliasing=False)
    return output

In [ ]:
def objective_function(z, ret_tuple = False):
    real_image = tifffile.imread("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/top_trenches_PC/trench_{}/T_{}.tif".format(
    str(np.random.randint(1,56)).zfill(2),
    str(np.random.randint(20,25)).zfill(3)
))
    real_image = real_image.astype(np.float64)/np.max(real_image)
    σ, trench_multiplier, cell_multiplier, background_multiplier = z
    expanded_scene = generate_PC_OPL(trench_multiplier,cell_multiplier,background_multiplier)
    kernel = get_phase_contrast_kernel(R, W, 50, scale, 5, σ)
    convolved_image = convolve_rescale(expanded_scene,kernel)[0:182,3:-3]
    #convolved_image = random_noise(convolved_image, mode="gaussian", mean=5,var=σ2,clip=False)
    convolved_image = match_histograms(convolved_image, real_image, multichannel=False)
    convolved_image = resize(convolved_image,real_image.shape,clip=False,preserve_range=False,anti_aliasing=None)
    convolved_image = convolved_image/np.max(convolved_image)
    ssim_real = ssim(convolved_image, real_image)
    intersection = return_intersection_between_image_hists(convolved_image, real_image, 100)
    #sims 
    convolved_image.shape += (1,)
    real_image.shape += (1,)
    _fsim = fsim(convolved_image,real_image)
    _issm = issm(convolved_image,real_image)
    _sam = sam(convolved_image,real_image)
    _sre = sre(convolved_image,real_image)
    objs = [ssim_real, 0.5*intersection, _fsim, _issm, _sam, _sre/20]
    if ret_tuple == False:
        return -np.linalg.norm(objs)
    else:
        return objs

progress = []
def callbackF(x, f, context):
    print(x)
    progress.append(x)
    
def callbackSHGo(x):
    print(x)
    progress.append(x)

In [ ]:
bounds = list(zip([min_sigma, 1.0, 0.2, 1], [20.0, 30.0, 5.0, 20.0]))

In [ ]:
ret = shgo(objective_function, bounds = np.array(bounds),callback=callbackSHGo,options={"minimize_every_iter":True})
ret = dual_annealing(objective_function, bounds = np.array(bounds),callback=callbackF,maxiter=1000,initial_temp=5*10**4,x0=ret.x)

In [ ]:
from skopt import gp_minimize
from skopt.callbacks import VerboseCallback
res = gp_minimize(objective_function,                  # the function to minimize
                  bounds,      # the bounds on each dimension of x
                  acq_func="gp_hedge",      # the acquisition function
                  n_calls=50,         # the number of evaluations of f
                  n_random_starts=5,  # the number of random initialization points
                 n_jobs = 1)   

In [ ]:
res.x

In [ ]:
def objective_function_get_image(z):
    σ, trench_multiplier, cell_multiplier, background_multiplier, = z
    expanded_scene = generate_PC_OPL(trench_multiplier,cell_multiplier,background_multiplier)
    kernel = get_phase_contrast_kernel(R, W, 50, scale, 5, σ)
    convolved_image = convolve_rescale(expanded_scene,kernel)[0:182,10:-10]
    #convolved_image = random_noise(convolved_image, mode="gaussian", mean=5,var=σ2,clip=False)
    convolved_image = match_histograms(convolved_image, real_image, multichannel=False)
    convolved_image = resize(convolved_image,real_image.shape,clip=False,preserve_range=False,anti_aliasing=None)
    convolved_image = convolved_image/np.max(convolved_image)
    ssim_real = ssim(convolved_image, real_image)
    intersection = return_intersection_between_image_hists(convolved_image, real_image, 100)
    return convolved_image

In [ ]:
real_image = tifffile.imread("/home/georgeos/Storage/Dropbox (Cambridge University)/PhD_Georgeos_Hardo/ML_based_segmentation_results/40x_Ph2_test_1.5/top_trenches_PC/trench_{}/T_{}.tif".format(
    str(np.random.randint(1,56)).zfill(2),
    str(np.random.randint(1,25)).zfill(3)
))
optimised_image = objective_function_get_image(ret.x)
noisy_image = random_noise(optimised_image, mode="gaussian", mean=5,var=0.0002,clip=False)
viewer = napari.view_image(noisy_image, rgb=False)



viewer = napari.view_image(real_image, rgb=False)

In [ ]:
real_image.shape

In [ ]:
convolved_outputs = []
for x in range(len(PC_OPLs)):
    output = cuconvolve(cp.array(PC_OPLs[x]),kernel)
    output = output.get()
    output = rescale(output, 1/1.75, anti_aliasing=False)[:364,:]
    output = random_noise(output, mode="gaussian", mean=5,var=0.85,clip=False)
    output_rescaled = match_histograms(output, real_phase[15][30:,:], multichannel=False)
    convolved_outputs.append(output_rescaled)
convolved_outputs = np.array(convolved_outputs)

In [ ]:
viewer = napari.view_image(convolved_outputs**2, rgb=False)

In [ ]:
    scalebar = ScaleBar(scale, 'um')
    plt.figure(figsize=(5,5))
    plt.gca().add_artist(scalebar)
    plt.imshow(output_rescaled,cmap="Greys_r")
    plt.axis("off")
    plt.title("Phase Contrast")
    plt.show()

In [ ]:
output.shape

In [ ]:
cuconvolve(cp.array(PC_OPLs[x]),kernel).shape